In [1]:
import pandas as pd
import psycopg2
import numpy as np
from sqlalchemy import create_engine, text as sql_text
from ydata_profiling import ProfileReport

# Projeto Analytics Engineering - Inside Airbnb

## PostgreSQL - Exportação dos dados "raw"

Conjunto de dados:  "Inside Airbnb"

Passo a Passo:
1. Vá no link https://insideairbnb.com/get-the-data
2. Procure por "Rio de Janeiro"
3. Baixe os 3 .csv: "Listing", "Reviews" e Calendar" 

0bs: Caso queira pular estes passos, basta utilizar os .csvs que já estão nessa pasta: https://drive.google.com/drive/folders/1pwCpVsMu5si7BX1lzWX3tipP3fAPuoX3?usp=drive_link

4. Crie uma pasta dentro desse repositorio como 'raw_data' e adicione os 3 .csv

### Conexão com PostgreSQL

In [2]:
# James:
# dbname='airbnb_project'
# user='postgres'
# host='localhost'
# password='admin'
# port='5432'

# Marcela: 
dbname='airbnb_project'
user='postgres'
host='localhost'
password='0125698741'
port='5433'

In [3]:
conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password, port=port)
db_params = {
    'dbname': dbname,
    'user': user,
    'host': host,
    'password': password,
    'port': port
}
db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'


### Criação das bases 'raw's
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") 

In [4]:
engine = create_engine(db_url)
#path = 'dbt/analytics_eng_airbnb/raw_data/'
path = 'raw_data/'
df_listings_raw = pd.read_csv(path + "listings.csv")
df_reviews_raw = pd.read_csv(path + "reviews.csv")
df_calendar_raw = pd.read_csv(path + "calendar.csv")

In [5]:
df_listings_raw.to_sql('listings_raw', engine, if_exists='replace', index=False)
df_reviews_raw.to_sql('reviews_raw', engine, if_exists='replace', index=False)
df_calendar_raw.to_sql('calendar_raw', engine, if_exists='replace', index=False)

595

In [5]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df

In [9]:
profile = ProfileReport(df_listings_raw, title="Listings Report")
profile.to_file("resultados_listings.html")
profile = ProfileReport(df_reviews_raw, title="Reviews Report", minimal=True)
profile.to_file("resultados_reviews.html")
profile = ProfileReport(df_calendar_raw, title="Calendar Report")
profile.to_file("resultados_calendar.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## DBT - Inicialização

Passo a Passo:
1) Instale o DBT
- python -m pip install dbt-postgres
- pip install dbt


2) Vá até sua pasta "dbt" e rode o comando:
3) dbt init analytics_eng_airbnb 

OBS: você pode pular os passos 2 e 3 utilizando a pasta "dbt" dentro do arquivo do Github


Atualize seus dados no 'profiles.yml'

# Camadas

## Bronze

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/bronze
- source.yml com os sources de "listing_raw", "reviews_raw", "calendar_raw"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_bronze", "reviews_bronze", "calendar_bronze". Nesse arquivo, executamos:
    - Verificamos se ID em "listings" é unique e not null
    - Adicionamos referencias de "foreign key" para listings_id

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _bronze para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models bronze

## Silver

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/silver
- source.yml com os sources de "listing_bronze", "reviews_bronze", "calendar_bronze"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_silver", "reviews_silver", "calendar_silver".
    - Aqui alem da limpeza mais bruta dos dados tambem começamos a selecionar quais colunas fazia sentido ou nao manter em nossas tabelas com relação aonde queriamos chega na camada gold, assim como aplicar filtros e condiçoes, tudo isso com o dbt. ex: Na tabela listing, na coluna reviews temos uma condição para que numero de reviws fosse maior que 0.

- arquivos .sql para cada uma das 3 tabelas. Nesses arquivos, executamos:
    - calendar_silver.sql
    - listings_silver.sql
    - reviews_silver.sql

Para executar as regras criadas e importar as tabelas _silver para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models silver

## Gold

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/gold
- source.yml com os sources de "listing_silver", "reviews_silver", "calendar_silver"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas tabelas para cada uma das visualizações imaginadas. Até o momento temos 5 tabelas:
  - most_available_by_weekday_gold: demonstra a disponibilidade de oferta de listagens para cada dia da semana. Esse indicador pode ser usado para prever os dias mais e menos movimentados para os hospedes e anfitriões.
  - price_avg_by_neighborhood_gold: explicita a média de preços em cada bairro da cidade do Rio de Janeiro. Esse ranking pode ajudar os anfitriões a escolherem os preços a serem praticados nos seus imoveis e a plataforma AirBnb a prever seus ganhos.
  - price_avg_by_type: similar a tabela anterior, porém agrupando os resultados por tipo de imovel (casa inteira, quarto, loft, etc). Suas aplicações práticas também são semelhantes as da tabela anterior.
  - comments_price_correlation_gold: A tabela comments_price_correlation_gold permite analisar se existe uma correlação entre o número de comentários que uma listagem recebe e seu preço. Por exemplo, listagens com preços mais altos podem receber mais comentários porque atraem mais hóspedes ou porque oferecem uma experiência de maior qualidade que leva a mais comentários. Alternativamente, listagens com preços mais baixos podem receber mais comentários porque são mais acessíveis para um maior número de hóspedes.
  - host_listing_analysis_gold: A tabela host_listing_analysis_gold permite analisar as características dos anfitriões e como elas se relacionam com o preço das hospedagens no airbnb. Por exemplo, você pode querer saber se anfitriões com mais hospedagens no airbnb tendem a ter preços mais altos ou mais baixos. Alternativamente, você pode querer saber se anfitriões com preços mais altos têm mais ou menos hospedagens no airbnb.


- arquivos .sql para cada uma das tabelas. Nesses arquivos, executamos:
    - host_listing_analysis_gold
    - comments_price_correlation_gold
    - price_avg_by_type
    - most_available_by_weekday_gold
    - price_avg_by_neighborhood_gold

Para executar as regras criadas e importar as tabelas _gold para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models gold


Nessa camada optamos por não executar testagens, com exceção de teste de redundância simples como verificar se campos de id são únicos e campos requeridos não são nulos.

## Todas as tabelas em execução no Postgres

![tabelas Postgres](./tabelasPGAdmin.jpeg)